In [3]:
import cv2
import numpy as np

def detect_trading_area(frame):
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Apply GaussianBlur to reduce noise
    blurred = cv2.GaussianBlur(gray, (9, 9), 2)
    # Detect edges using Canny
    edges = cv2.Canny(blurred, 50, 150)
    
    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        if len(approx) == 4:  # Detect rectangles
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = w / float(h)
            if 0.95 <= aspect_ratio <= 1.05 and cv2.isContourConvex(approx):
                return x, y, w, h  # Return bounding box of the trading area

    return None

def detect_circles(frame, roi):
    x, y, w, h = roi
    roi_frame = frame[y:y+h, x:x+w]
    gray_roi = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2GRAY)
    # Detect circles using HoughCircles
    circles = cv2.HoughCircles(
        gray_roi, cv2.HOUGH_GRADIENT, dp=1, minDist=20,
        param1=50, param2=30, minRadius=5, maxRadius=50
    )
    return circles, roi_frame

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    trading_area = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if trading_area is None:
            trading_area = detect_trading_area(frame)
            if trading_area:
                print("Trading area detected.")
            else:
                print("No trading area detected. Skipping frame.")
                continue

        # Detect circles in the trading area
        circles, roi_frame = detect_circles(frame, trading_area)
        
        if circles is not None:
            circles = np.uint16(np.around(circles))
            for circle in circles[0, :]:
                center = (circle[0], circle[1])
                radius = circle[2]
                # Draw the circle
                cv2.circle(roi_frame, center, radius, (0, 255, 0), 2)
                cv2.circle(roi_frame, center, 2, (0, 0, 255), 3)
            # Display message on video
            cv2.putText(frame, "Trading is happening!", (50, 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Draw the trading area on the original frame
        x, y, w, h = trading_area
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.imshow("Trading Area Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Example usage
process_video('./materials/E2-trading,lis,wilk.MP4')


Trading area detected.


In [9]:
import cv2
import numpy as np

def detect_black_area(frame):
    """
    Detect the black trading area in the top-right corner of the frame.
    """
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to isolate dark areas
    _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)

    # Focus only on the top-right part of the frame (trading area is expected here)
    height, width = thresh.shape
    roi = thresh[0:height//2, width//2:]  # Top-right half

    # Find contours in the ROI
    contours, _ = cv2.findContours(roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        # Filter for large enough contours
        area = cv2.contourArea(contour)
        if area > 500:  # Minimum area to qualify as a trading area
            # Get bounding box
            x, y, w, h = cv2.boundingRect(contour)
            return x + width//2, y, w, h  # Adjust x-coordinates to the full frame

    return None

def detect_circles(frame, roi):
    """
    Detect circular shapes within the trading area (ROI).
    """
    x, y, w, h = roi
    roi_frame = frame[y:y+h, x:x+w]
    gray_roi = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2GRAY)

    # Use HoughCircles to detect circles
    circles = cv2.HoughCircles(
        gray_roi, cv2.HOUGH_GRADIENT, dp=1.2, minDist=20,
        param1=50, param2=30, minRadius=5, maxRadius=50
    )

    return circles, roi_frame

def process_video(video_path):
    """
    Process the video to detect the trading area and identify trading events.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    while True:
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            break

        # Detect the black trading area
        trading_area = detect_black_area(frame)
        if trading_area:
            x, y, w, h = trading_area

            # Detect circles in the trading area
            circles, roi_frame = detect_circles(frame, trading_area)

            # Draw the detected trading area
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

            # If circles are detected, mark them and display the trading message
            if circles is not None:
                circles = np.uint16(np.around(circles))
                for circle in circles[0, :]:
                    cx, cy, radius = circle
                    # Draw the circle on the ROI
                    cv2.circle(roi_frame, (cx, cy), radius, (0, 255, 0), 2)
                    cv2.circle(roi_frame, (cx, cy), 2, (0, 0, 255), 3)

                # Display the message for trading detection
                cv2.putText(frame, "Trading is happening!", (50, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display the frame with detection
        cv2.imshow("Trading Detection", frame)

        # Break on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

# Example usage
process_video('./materials/E2-trading,lis,wilk.MP4')


In [ ]:
import cv2
import numpy as np
from collections import deque

# Buffer to track detected circles across frames
circle_buffer = deque(maxlen=10)  # Store the last 10 frames

def detect_black_area(frame):
    """
    Detect the top part of the black trading area in the frame.
    """
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to isolate dark areas
    _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)

    # Focus on the top-right part of the frame (trading area is expected here)
    height, width = thresh.shape
    roi = thresh[0:height//2, width//2:]  # Top-right half

    # Find contours in the ROI
    contours, _ = cv2.findContours(roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        # Filter for large enough contours
        area = cv2.contourArea(contour)
        if area > 500:  # Minimum area to qualify as a trading area
            # Get bounding box
            x, y, w, h = cv2.boundingRect(contour)
            return x + width//2, y, w, h  # Adjust x-coordinates to the full frame

    return None

def detect_circles(frame, roi):
    """
    Detect circular shapes within the trading area (ROI).
    """
    x, y, w, h = roi
    roi_frame = frame[y:y+h, x:x+w]
    gray_roi = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2GRAY)

    # Use HoughCircles to detect circles
    circles = cv2.HoughCircles(
        gray_roi, cv2.HOUGH_GRADIENT, dp=1.2, minDist=20,
        param1=50, param2=30, minRadius=10, maxRadius=50  # Tuned for tokens
    )

    if circles is not None:
        # Filter out dice by ensuring circularity and size constraints
        valid_circles = []
        for circle in np.uint16(np.around(circles))[0, :]:
            cx, cy, radius = circle
            # Circularity and size check
            if 15 <= radius <= 35:  # Tokens typically fall within this size range
                valid_circles.append((cx, cy, radius))
        return valid_circles, roi_frame

    return None, roi_frame

def is_trading(circles):
    """
    Check if trading is happening by ensuring circles are detected consistently.
    """
    global circle_buffer
    circle_buffer.append(len(circles) if circles else 0)

    # Check if circles have been detected consistently for 5+ frames
    return sum(1 for count in circle_buffer if count > 0) >= 5

def process_video(video_path):
    """
    Process the video to detect the trading area and identify trading events.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    while True:
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            break

        # Detect the black trading area
        trading_area = detect_black_area(frame)
        if trading_area:
            x, y, w, h = trading_area

            # Detect circles in the trading area
            circles, roi_frame = detect_circles(frame, trading_area)

            # Draw the detected trading area
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

            # If circles are detected, mark them
            if circles:
                for cx, cy, radius in circles:
                    # Draw the circle on the ROI
                    cv2.circle(roi_frame, (cx, cy), radius, (0, 255, 0), 2)
                    cv2.circle(roi_frame, (cx, cy), 2, (0, 0, 255), 3)

            # Check if trading is happening
            if circles and is_trading(circles):
                # Display the message for trading detection
                cv2.putText(frame, "Trading is happening!", (50, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display the frame with detection
        cv2.imshow("Trading Detection", frame)

        # Break on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

# Example usage
process_video('./materials/E2-trading,lis,wilk.MP4')


In [14]:
import cv2
import numpy as np
from collections import deque

# Buffer to track detected circles across frames
circle_buffer = deque(maxlen=30)  # Store the last 10 frames

def detect_black_area(frame):
    """
    Detect the trading area in the top-right corner of the frame.
    Ensures the detected area is sufficiently large and matches expected proportions.
    """
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to isolate dark areas
    _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)

    # Focus on the top-right part of the frame (trading area is expected here)
    height, width = thresh.shape
    roi = thresh[0:height//2, width//2:]  # Top-right half

    # Find contours in the ROI
    contours, _ = cv2.findContours(roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        # Filter for sufficiently large contours
        area = cv2.contourArea(contour)
        if area < 1000:  # Minimum size to consider as the trading area
            continue

        # Get bounding box of the contour
        x, y, w, h = cv2.boundingRect(contour)

        # Ensure the area matches the expected proportions
        aspect_ratio = w / float(h)
        if 0.8 <= aspect_ratio <= 1.2 and area > 1500:
            return x + width//2, y, w, h  # Adjust x-coordinates to the full frame

    return None

def detect_circles(frame, roi):
    """
    Detect circular shapes within the trading area (ROI).
    Uses circularity and size constraints to filter out dice.
    """
    x, y, w, h = roi
    roi_frame = frame[y:y+h, x:x+w]
    gray_roi = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2GRAY)

    # Use HoughCircles to detect circles
    circles = cv2.HoughCircles(
        gray_roi, cv2.HOUGH_GRADIENT, dp=1.2, minDist=20,
        param1=50, param2=30, minRadius=10, maxRadius=35  # Tuned for tokens
    )

    if circles is not None:
        # Filter circles based on size and circularity
        valid_circles = []
        for circle in np.uint16(np.around(circles))[0, :]:
            cx, cy, radius = circle
            if 15 <= radius <= 35:  # Tokens fall within this size range
                # Compute circularity: (4 * π * area) / perimeter^2
                contour_area = np.pi * (radius**2)
                perimeter = 2 * np.pi * radius
                circularity = (4 * np.pi * contour_area) / (perimeter**2)
                if circularity > 0.7:  # Ensure it’s circular
                    valid_circles.append((cx, cy, radius))
        return valid_circles, roi_frame

    return None, roi_frame

def is_trading(circles):
    """
    Check if trading is happening by ensuring circles are detected consistently.
    """
    global circle_buffer
    circle_buffer.append(len(circles) if circles else 0)

    # Check if circles have been detected consistently for 5+ frames
    return sum(1 for count in circle_buffer if count > 0) >= 5

def process_video(video_path):
    """
    Process the video to detect the trading area and identify trading events.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    while True:
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            break

        # Detect the black trading area
        trading_area = detect_black_area(frame)
        if trading_area:
            x, y, w, h = trading_area

            # Detect circles in the trading area
            circles, roi_frame = detect_circles(frame, trading_area)

            # Draw the detected trading area
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

            # If circles are detected, mark them
            if circles:
                for cx, cy, radius in circles:
                    # Draw the circle on the ROI
                    cv2.circle(roi_frame, (cx, cy), radius, (0, 255, 0), 2)
                    cv2.circle(roi_frame, (cx, cy), 2, (0, 0, 255), 3)

            # Check if trading is happening
            if circles and is_trading(circles):
                # Display the message for trading detection
                cv2.putText(frame, "Trading is happening!", (50, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display the frame with detection
        cv2.imshow("Trading Detection", frame)

        # Break on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

# Example usage
process_video('./materials/E2-trading,lis,wilk.MP4')


In [16]:
import cv2
import numpy as np
from collections import deque

# Buffer to track detected tokens across frames
circle_buffer = deque(maxlen=10)  # Store the last 10 frames


def detect_black_area(frame):
    """
    Detect the black trading area in the top-right corner of the frame.
    Uses contours and mean intensity filtering to ensure robustness.
    """
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to isolate dark areas
    _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)

    # Focus on the top-right part of the frame (trading area is expected here)
    height, width = thresh.shape
    roi = thresh[0:height//2, width//2:]  # Top-right half

    # Find contours in the ROI
    contours, _ = cv2.findContours(roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        # Filter for sufficiently large contours
        area = cv2.contourArea(contour)
        if area < 800:  # Minimum size to consider as the trading area
            continue

        # Get bounding box of the contour
        x, y, w, h = cv2.boundingRect(contour)

        # Check the mean intensity of the area to confirm it's black
        mean_intensity = np.mean(gray[y:y+h, x+width//2:x+width//2+w])
        if mean_intensity < 50:  # Black areas have low intensity
            return x + width//2, y, w, h  # Adjust x-coordinates to the full frame

    return None


def detect_circles(frame, roi):
    """
    Detect circular shapes within the trading area (ROI).
    Uses circularity and size constraints to filter out dice.
    """
    x, y, w, h = roi
    roi_frame = frame[y:y+h, x:x+w]
    gray_roi = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2GRAY)

    # Use HoughCircles to detect circles
    circles = cv2.HoughCircles(
        gray_roi, cv2.HOUGH_GRADIENT, dp=1.2, minDist=15,
        param1=50, param2=20, minRadius=10, maxRadius=40  # Tuned for tokens
    )

    if circles is not None:
        # Filter circles based on size and circularity
        valid_circles = []
        for circle in np.uint16(np.around(circles))[0, :]:
            cx, cy, radius = circle
            if 15 <= radius <= 35:  # Tokens fall within this size range
                valid_circles.append((cx, cy, radius))
        return valid_circles, roi_frame

    return None, roi_frame


def is_trading(circles):
    """
    Check if trading is happening by ensuring circles are detected consistently.
    """
    global circle_buffer
    circle_buffer.append(len(circles) if circles else 0)

    # Check if circles have been detected consistently for 5+ frames
    return sum(1 for count in circle_buffer if count > 0) >= 5


def process_video(video_path):
    """
    Process the video to detect the trading area and identify trading events.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    while True:
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            break

        # Detect the black trading area
        trading_area = detect_black_area(frame)
        if trading_area:
            x, y, w, h = trading_area

            # Detect circles in the trading area
            circles, roi_frame = detect_circles(frame, trading_area)

            # Draw the detected trading area
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

            # If circles are detected, mark them
            if circles:
                for cx, cy, radius in circles:
                    # Draw the circle on the ROI
                    cv2.circle(roi_frame, (cx, cy), radius, (0, 255, 0), 2)
                    cv2.circle(roi_frame, (cx, cy), 2, (0, 0, 255), 3)

            # Check if trading is happening
            if circles and is_trading(circles):
                # Display the message for trading detection
                cv2.putText(frame, "Trading is happening!", (50, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display the frame with detection
        cv2.imshow("Trading Detection", frame)

        # Break on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()


# Example usage
process_video('./materials/E1-trading,wilkx2.MP4')

In [17]:
import cv2
import numpy as np
from collections import deque

# Buffer to track tokens detected near the black area for consecutive frames
trading_buffer = deque(maxlen=10)

def detect_animal_tokens(frame):
    """
    Detect animal tokens based on their distinct colors in the HSV color space.
    Returns a list of bounding boxes around detected tokens.
    """
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define color ranges for animal tokens (tune as needed)
    color_ranges = [
        {"lower": (25, 50, 50), "upper": (35, 255, 255)},  # Yellow tokens
        {"lower": (100, 50, 50), "upper": (130, 255, 255)},  # Blue tokens
        {"lower": (0, 50, 50), "upper": (10, 255, 255)},  # Red tokens
        # Add more color ranges if needed
    ]

    tokens = []
    for color in color_ranges:
        mask = cv2.inRange(hsv, color["lower"], color["upper"])
        # Find contours for the masked regions
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            area = cv2.contourArea(contour)
            if area > 300:  # Filter small noise
                x, y, w, h = cv2.boundingRect(contour)
                tokens.append((x, y, w, h))

    return tokens


def is_near_black_area(frame, token):
    """
    Check if the given token is near a black region.
    """
    x, y, w, h = token
    # Expand the ROI slightly around the token
    margin = 10
    roi = frame[max(0, y-margin):y+h+margin, max(0, x-margin):x+w+margin]

    # Convert ROI to grayscale and check the mean intensity
    gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    mean_intensity = np.mean(gray_roi)
    return mean_intensity < 50  # Black areas have low intensity


def process_video(video_path):
    """
    Process the video to detect animal tokens and identify trading events.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    while True:
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            break

        # Detect animal tokens
        tokens = detect_animal_tokens(frame)

        # Check if any token is near a black area
        trading_detected = False
        for token in tokens:
            if is_near_black_area(frame, token):
                trading_detected = True
                break

        # Update the trading buffer
        trading_buffer.append(trading_detected)

        # Confirm trading if detected for 10 consecutive frames
        if sum(trading_buffer) >= 10:
            cv2.putText(frame, "Trading is happening!", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Draw detected tokens
        for token in tokens:
            x, y, w, h = token
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        # Display the frame
        cv2.imshow("Trading Detection", frame)

        # Break on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()


# Example usage
process_video('./materials/E1-trading,wilkx2.MP4')


In [19]:
import cv2
import numpy as np
from scipy.stats import mode

def detect_black_area(frame):
    """
    Detect the black trading area in the frame.
    This method assumes that the black area is the darkest region in the frame.
    """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 500:  # Minimum size of the trading area
            x, y, w, h = cv2.boundingRect(contour)
            return x, y, w, h

    return None

def get_dominant_color(frame, mask):
    """
    Calculate the dominant color inside the masked region of the frame.
    """
    mask_flat = mask.flatten()
    frame_flat = frame.reshape((-1, 3))
    color_mask = frame_flat[mask_flat == 255]

    if color_mask.size == 0:
        return (0, 0, 0)

    dominant_color = mode(color_mask, axis=0, keepdims=False).mode
    return tuple(map(int, dominant_color))

def detect_circles_and_colors(frame, trading_area):
    """
    Detect circular tokens and their colors in the frame.
    """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (9, 9), 2)

    # Use HoughCircles to detect circles
    circles = cv2.HoughCircles(
        blurred, cv2.HOUGH_GRADIENT, dp=1.2, minDist=30,
        param1=50, param2=30, minRadius=10, maxRadius=40
    )

    detected_tokens = []
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for circle in circles[0, :]:
            cx, cy, r = circle

            # Create a mask for the circle
            mask = np.zeros_like(gray, dtype=np.uint8)
            cv2.circle(mask, (cx, cy), r, 255, -1)

            # Get the dominant color within the circle
            dominant_color = get_dominant_color(frame, mask)

            # Check if the circle is near the trading area
            x, y, w, h = trading_area
            if x <= cx <= x + w and y <= cy <= y + h:
                detected_tokens.append((cx, cy, r, dominant_color))

    return detected_tokens

def process_video(video_path):
    """
    Process the video to detect tokens and check for trading events.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Detect the black trading area
        trading_area = detect_black_area(frame)
        if trading_area:
            x, y, w, h = trading_area
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # Detect circles (tokens) and their colors
            tokens = detect_circles_and_colors(frame, trading_area)

            for cx, cy, r, color in tokens:
                # Draw the circle and the dominant color
                cv2.circle(frame, (cx, cy), r, (0, 255, 0), 2)
                cv2.circle(frame, (cx, cy), 2, (0, 0, 255), 3)
                cv2.putText(frame, f"{color}", (cx - 20, cy - 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        cv2.imshow("Trading Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Example usage
# process_video("path_to_your_video.mp4")

# Example usage
process_video('./materials/E1-trading,wilkx2.MP4')


In [20]:
import cv2
import numpy as np

def preprocess_frame(frame):
    """
    Preprocess the frame to enhance contrast for token detection.
    """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    return blurred

def detect_circles(frame):
    """
    Detect circular tokens using HoughCircles.
    """
    blurred = preprocess_frame(frame)

    # Detect circles using HoughCircles
    circles = cv2.HoughCircles(
        blurred, cv2.HOUGH_GRADIENT, dp=1.2, minDist=30,
        param1=50, param2=30, minRadius=10, maxRadius=40
    )

    if circles is not None:
        return np.uint16(np.around(circles))
    return None

def get_circle_color(frame, circle):
    """
    Get the dominant color inside the detected circle.
    """
    cx, cy, radius = circle
    mask = np.zeros(frame.shape[:2], dtype=np.uint8)
    cv2.circle(mask, (cx, cy), radius, 255, -1)

    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    masked_pixels = hsv_frame[mask == 255]

    if len(masked_pixels) > 0:
        # Get the average color in HSV space
        avg_color = np.mean(masked_pixels, axis=0)
        return avg_color
    return None

def is_token_color(color):
    """
    Determine if the given color matches token colors.
    """
    # Define HSV ranges for token colors
    token_color_ranges = [
        {"lower": (20, 100, 100), "upper": (30, 255, 255)},  # Example: Yellow
        {"lower": (100, 100, 100), "upper": (140, 255, 255)},  # Example: Blue
        {"lower": (0, 100, 100), "upper": (10, 255, 255)},  # Example: Red
    ]

    for color_range in token_color_ranges:
        if all(color_range["lower"][i] <= color[i] <= color_range["upper"][i] for i in range(3)):
            return True
    return False

def process_video(video_path):
    """
    Process the video to detect tokens and identify trading events.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Detect circles
        circles = detect_circles(frame)

        if circles is not None:
            for circle in circles[0, :]:
                cx, cy, radius = circle

                # Get the dominant color inside the circle
                color = get_circle_color(frame, (cx, cy, radius))

                if color is not None and is_token_color(color):
                    # Draw the detected token
                    cv2.circle(frame, (cx, cy), radius, (0, 255, 0), 2)
                    cv2.circle(frame, (cx, cy), 2, (0, 0, 255), 3)

                    # Display token detection
                    cv2.putText(frame, "Token", (cx - 20, cy - 20),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Display the frame
        cv2.imshow("Token Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

process_video('./materials/E1-trading,wilkx2.MP4')


---------

In [4]:
import cv2
import numpy as np
import time

In [15]:
import cv2
import numpy as np
from collections import Counter
import time

def reorder_corners(corners):
    sorted_corners = sorted(corners, key=lambda x: x[0])
    top_left, top_right = sorted_corners[:2]
    bottom_left, bottom_right = sorted_corners[2:]

    if top_left[1] > top_right[1]:
        top_left, top_right = top_right, top_left

    if bottom_left[1] > bottom_right[1]:
        bottom_left, bottom_right = bottom_right, bottom_left

    return np.array([top_left, top_right, bottom_left, bottom_right])

def detect_board(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (9, 9), 1)
    edges = cv2.Canny(gray, 50, 150)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    edges_closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)
    contours, _ = cv2.findContours(edges_closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        epsilon = 0.05 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        if len(approx) == 4:
            approx = reorder_corners(approx.reshape(4, 2))
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = w / float(h)
            area = cv2.contourArea(contour)
            if 0.8 <= aspect_ratio <= 1.2 and area > 50000:
                return approx, x, y, w, h
    return None, None, None, None, None

def create_tracker(tracker_type="CSRT"):
    if tracker_type == "CSRT":
        return cv2.TrackerCSRT_create()
    elif tracker_type == "KCF":
        return cv2.TrackerKCF_create()
    return None

def draw_bbox_with_area(frame, bbox, color=(255, 255, 255)):
    p1 = (int(bbox[0]), int(bbox[1]))
    p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
    cv2.rectangle(frame, p1, p2, color, 2, 1)
    text_position = (p1[0], max(0, p1[1] - 10))
    cv2.putText(frame, "Dice", text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)

def get_dominant_color(image, mask):
    colors = image[mask == 255]
    if len(colors) == 0:
        return (0, 0, 0)

    colors = [tuple(color) for color in colors]
    most_common_color = Counter(colors).most_common(1)[0][0]

    return most_common_color

def detect_circles_and_colors(frame, hsv_frame, thresholds):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (9, 9), 2)
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1.2, minDist=30, param1=200, param2=100, minRadius=10, maxRadius=100)

    detected_tokens = []

    if circles is not None:
        circles = np.uint16(np.around(circles))
        for circle in circles[0, :]:
            x, y, r = circle

            # Create a circular mask for the detected circle
            mask = np.zeros_like(gray, dtype=np.uint8)
            cv2.circle(mask, (x, y), r, 255, -1)

            # Get dominant color inside the circle
            dominant_color = get_dominant_color(frame, mask)

            # Determine animal type based on thresholds
            for animal, (lower, upper) in thresholds.items():
                if np.all(lower <= dominant_color) and np.all(dominant_color <= upper):
                    detected_tokens.append((animal, (x, y, r), dominant_color))

    return detected_tokens

def detect_circles_and_dice_in_video(video_path, thresholds):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Detect animal tokens
        detected_tokens = detect_circles_and_colors(frame, hsv_frame, thresholds)
        for animal, (x, y, r), dominant_color in detected_tokens:
            color = (0, 255, 0)
            cv2.circle(frame, (x, y), r, color, 2)
            label = f"{animal}"
            cv2.putText(frame, label, (x - r, y - r - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        cv2.imshow("Circles and Colors", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

animal_color_thresholds = {
    "rabbit": (np.array([0, 50, 50]), np.array([10, 255, 255])),
    "sheep": (np.array([20, 50, 50]), np.array([30, 255, 255])),
    "pig": (np.array([30, 50, 50]), np.array([40, 255, 255])),
    "cow": (np.array([0, 0, 50]), np.array([180, 50, 255])),
    "horse": (np.array([50, 50, 50]), np.array([70, 255, 255]))
}

detect_circles_and_dice_in_video('./materials/E1.MP4', animal_color_thresholds)


In [17]:
import cv2
import numpy as np
from collections import Counter, defaultdict

# Helper function to calculate the dominant color

def get_dominant_color(image, mask):
    colors = image[mask == 255]
    if len(colors) == 0:
        return (0, 0, 0)

    colors = [tuple(color) for color in colors]
    most_common_color = Counter(colors).most_common(1)[0][0]

    return most_common_color

# Function to detect circles and stabilize color classification
def detect_circles_and_colors_stable(video_path, thresholds, stability_window=5):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    circle_history = defaultdict(list)  # Store recent classifications per circle
    stable_results = {}  # Final stabilized results

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (9, 9), 2)
        circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1.2, minDist=50, param1=70, param2=30, minRadius=10, maxRadius=100)

        if circles is not None:
            circles = np.uint16(np.around(circles))
            for i, circle in enumerate(circles[0, :]):
                x, y, r = circle

                # Create a circular mask for the detected circle
                mask = np.zeros_like(gray, dtype=np.uint8)
                cv2.circle(mask, (x, y), r, 255, -1)

                # Get dominant color inside the circle
                dominant_color = get_dominant_color(frame, mask)

                # Determine animal type based on thresholds
                detected_animal = None
                for animal, (lower, upper) in thresholds.items():
                    if np.all(lower <= dominant_color) and np.all(dominant_color <= upper):
                        detected_animal = animal
                        break

                if detected_animal:
                    circle_history[i].append(detected_animal)

                    # Keep only the last 'stability_window' classifications
                    if len(circle_history[i]) > stability_window:
                        circle_history[i].pop(0)

                    # Determine the most frequent classification
                    most_stable_animal = Counter(circle_history[i]).most_common(1)[0][0]
                    stable_results[i] = most_stable_animal

                    # Draw the circle and label
                    color = (0, 255, 0)
                    cv2.circle(frame, (x, y), r, color, 2)
                    cv2.putText(frame, most_stable_animal, (x - r, y - r - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        cv2.imshow("Stabilized Animal Tokens", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

animal_color_thresholds = {
    "rabbit": (np.array([0, 50, 50]), np.array([10, 255, 255])),
    "sheep": (np.array([20, 50, 50]), np.array([30, 255, 255])),
    "pig": (np.array([30, 50, 50]), np.array([40, 255, 255])),
    "cow": (np.array([0, 0, 50]), np.array([180, 50, 255])),
    "horse": (np.array([50, 50, 50]), np.array([70, 255, 255]))
}

detect_circles_and_colors_stable('./materials/E2.MP4', animal_color_thresholds)


In [28]:
import cv2
import numpy as np
from collections import Counter, defaultdict

# Helper function to calculate the dominant color

def get_dominant_color(image, mask):
    colors = image[mask == 255]
    if len(colors) == 0:
        return (0, 0, 0)

    colors = [tuple(color) for color in colors]
    most_common_color = Counter(colors).most_common(1)[0][0]

    return most_common_color

# Function to detect circles and stabilize color classification

def compute_circle_size_and_tolerance():
    # Define circle size and tolerance (example: derived from your board logic)
    circle_size = 50  # Average circle radius in pixels
    tolerance = 0.07 * circle_size  # Example tolerance
    return circle_size, tolerance

def detect_circles_and_colors_stable(video_path, thresholds, stability_window=5):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    circle_size, tolerance = compute_circle_size_and_tolerance()
    circle_history = defaultdict(list)  # Store recent classifications per circle
    stable_results = {}  # Final stabilized results

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (9, 9), 2)
        circles = cv2.HoughCircles(
            blurred, cv2.HOUGH_GRADIENT, dp=1.2, minDist=50,
            param1=70, param2=30,
            minRadius=int(circle_size - tolerance),
            maxRadius=int(circle_size + tolerance)
        )

        if circles is not None:
            circles = np.uint16(np.around(circles))
            for i, circle in enumerate(circles[0, :]):
                x, y, r = circle

                # Create a circular mask for the detected circle
                mask = np.zeros_like(gray, dtype=np.uint8)
                cv2.circle(mask, (x, y), r, 255, -1)

                # Get dominant color inside the circle
                dominant_color = get_dominant_color(frame, mask)

                # Determine animal type based on thresholds
                detected_animal = None
                for animal, (lower, upper) in thresholds.items():
                    if np.all(lower <= dominant_color) and np.all(dominant_color <= upper):
                        detected_animal = animal
                        break

                if detected_animal:
                    circle_history[i].append(detected_animal)

                    # Keep only the last 'stability_window' classifications
                    if len(circle_history[i]) > stability_window:
                        circle_history[i].pop(0)

                    # Determine the most frequent classification
                    most_stable_animal = Counter(circle_history[i]).most_common(1)[0][0]
                    stable_results[i] = most_stable_animal

                    # Draw the circle and label
                    color = (0, 255, 0)
                    cv2.circle(frame, (x, y), r, (0, 255, 0), 2)
                    cv2.putText(frame, most_stable_animal, (x - r, y - r - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                else:
                    # Draw unclassified circles in red
                    cv2.circle(frame, (x, y), r, (0, 0, 255), 2)

        cv2.imshow("Stabilized Animal Tokens", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

animal_color_thresholds = {
    "rabbit": (np.array([0, 50, 50]), np.array([10, 255, 255])),
    "sheep": (np.array([20, 50, 50]), np.array([30, 255, 255])),
    "pig": (np.array([30, 50, 50]), np.array([40, 255, 255])),
    "cow": (np.array([0, 0, 50]), np.array([180, 50, 255])),
    "horse": (np.array([50, 50, 50]), np.array([70, 255, 255]))
}

detect_circles_and_colors_stable('./materials/EExtra.MP4', animal_color_thresholds)

# detect_circles_and_colors_debug('./materials/E2.MP4')


detecting just the cirkles- tokens without naming etc

In [30]:
import cv2
import numpy as np

# Function to detect circles in a video

def compute_circle_size_and_tolerance():
    # Define circle size and tolerance (example: derived from your board logic)
    circle_size = 50  # Average circle radius in pixels
    tolerance = 0.07 * circle_size  # Example tolerance
    return circle_size, tolerance

def detect_circles(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    circle_size, tolerance = compute_circle_size_and_tolerance()

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (9, 9), 2)
        circles = cv2.HoughCircles(
            blurred, cv2.HOUGH_GRADIENT, dp=1.2, minDist=50,
            param1=70, param2=30,
            minRadius=int(circle_size - tolerance),
            maxRadius=int(circle_size + tolerance)
        )

        if circles is not None:
            circles = np.uint16(np.around(circles))
            for circle in circles[0, :]:
                x, y, r = circle
                # Draw the circle on the frame
                cv2.circle(frame, (x, y), r, (0, 255, 0), 2)

        cv2.imshow("Detected Circles", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# # Call the function to detect circles
# detect_circles('./materials/E1.MP4')
# detect_circles('./materials/E2.MP4')
# detect_circles('./materials/EExtra.MP4')
detect_circles('./materials/H1.MP4')

more stabe version:

In [37]:
import cv2
import numpy as np

# Function to compute circle size and tolerance
def compute_circle_size_and_tolerance():
    circle_size = 50  # Average circle radius in pixels
    tolerance = 0.07 * circle_size  # Example tolerance
    return circle_size, tolerance

# Function to stabilize circle detection with exponential moving average
def stabilize_circles(new_circles, previous_circles, alpha=0.98, distance_threshold=20):
    if previous_circles is None:
        return new_circles

    stabilized_circles = []
    for new_circle in new_circles:
        x_new, y_new, r_new = new_circle
        matched = False
        for prev_circle in previous_circles:
            x_prev, y_prev, r_prev = prev_circle
            distance = np.sqrt((x_new - x_prev)**2 + (y_new - y_prev)**2)
            if distance < distance_threshold:
                # Apply exponential moving average for stabilization
                x_stabilized = alpha * x_prev + (1 - alpha) * x_new
                y_stabilized = alpha * y_prev + (1 - alpha) * y_new
                r_stabilized = alpha * r_prev + (1 - alpha) * r_new
                stabilized_circles.append((x_stabilized, y_stabilized, r_stabilized))
                matched = True
                break
        if not matched:
            stabilized_circles.append(new_circle)

    return stabilized_circles

# Function to detect circles in a video
def detect_circles(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    circle_size, tolerance = compute_circle_size_and_tolerance()
    previous_circles = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (9, 9), 2)
        circles = cv2.HoughCircles(
            blurred, cv2.HOUGH_GRADIENT, dp=1.2, minDist=50,
            param1=70, param2=30,
            minRadius=int(circle_size - tolerance),
            maxRadius=int(circle_size + tolerance)
        )

        if circles is not None:
            circles = np.uint16(np.around(circles))[0, :]
            stabilized_circles = stabilize_circles(circles, previous_circles)
            previous_circles = stabilized_circles

            for circle in stabilized_circles:
                x, y, r = map(int, circle)
                # Draw the circle on the frame
                cv2.circle(frame, (x, y), r, (0, 255, 0), 2)

        cv2.imshow("Stabilized Circles", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# # Call the function to detect circles
# detect_circles('./materials/E1.MP4')
detect_circles('./materials/E2.MP4')
# detect_circles('./materials/EExtra.MP4')
# detect_circles('./materials/H1.MP4')

C:\Users\mmart\AppData\Local\Temp\ipykernel_31316\3138658194.py:21: RuntimeWarning: overflow encountered in scalar subtract
  distance = np.sqrt((x_new - x_prev)**2 + (y_new - y_prev)**2)
C:\Users\mmart\AppData\Local\Temp\ipykernel_31316\3138658194.py:21: RuntimeWarning: invalid value encountered in sqrt
  distance = np.sqrt((x_new - x_prev)**2 + (y_new - y_prev)**2)
